In [3]:
import os
import torch
import torchtext
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch import nn
from torch.nn import functional as F

In [3]:
data = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
data[data['threat'] == 1].count()

id               478
comment_text     478
toxic            478
severe_toxic     478
obscene          478
threat           478
insult           478
identity_hate    478
dtype: int64

In [7]:
X = data['comment_text']
y = data[data.columns[2:]].values

In [6]:
y = np.array(y)
y.shape

(159571, 6)

In [5]:
X = iter(X)
y = iter(y)

In [53]:
X = np.array(X)
X.shape

(159571,)

In [54]:
y = np.array(y)
y.shape

(159571, 6)

In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [7]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
vocab = build_vocab_from_iterator(yield_tokens(X), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [8]:
vocab(['here', 'is', 'an', 'example'])

[73, 11, 39, 314]

In [9]:
len(vocab)

243144

In [8]:
text_pipeline = lambda x: vocab(tokenizer(x))

In [55]:
np.array(text_pipeline(X[0])).shape

(60,)

In [46]:
y[0].shape

(6,)

In [9]:
class comment_dataset(Dataset):
    def __init__(self, X, y, vocab, tokenizer, max_words = 2000):
        self.data = np.array(X)
        self.labels = np.array(y)
        self.max_words = max_words
    def __getitem__(self, index=0):
        text_data = self.data[index]
        labels = self.labels[index, :]
        text_data = vocab(tokenizer(text_data))
        text_data = text_data +([0]* (self.max_words-len(text_data))) if len(text_data)<self.max_words else text_data[:self.max_words]
        #text_data = text_pipeline(text_data)
        return torch.tensor(text_data, dtype = torch.int32), torch.tensor(labels)

    def __len__(self):
        return self.data.shape[0]

In [9]:
max_words = 25

In [12]:
def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y)

In [52]:
x = vocab(tokenizer(X[0]))
x = x +([0]* (max_words-len(x))) if len(x)<max_words else x[:max_words]
#np.array(x, dtype = np.int32).shape
torch.tensor(x, dtype = torch.int32)
#x

tensor([  666,    84,     2,   139,   141,   184,    38,   662,  4899, 11397,
         1286,    95,   328,    26,    56,  2218,     8,    30, 11469,     3,
           60,  6855,    20,    70,  2784], dtype=torch.int32)

In [7]:
tokenizer = get_tokenizer('basic_english')

In [14]:
dataset = comment_dataset(X, y, vocab = vocab, tokenizer = tokenizer)
val_size = int(0.2*len(dataset))
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size = 34
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size = batch_size, shuffle=True)

In [16]:
len(val_ds)

31914

In [11]:
for data, labels in train_dataloader:
    print(data.size())
    print(labels.size())
    break

torch.Size([34, 2000])
torch.Size([34, 6])


In [4]:

embed_len = 50
hidden_dim = 128
n_layers=2

class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.seq = nn.Sequential(nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len))
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size = embed_len, hidden_size = hidden_dim, num_layers = n_layers, batch_first = True, bidirectional = True)
        #self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Sequential(nn.Linear(2 * hidden_dim, 128),
                                    nn.ReLU(),
                                    # nn.Linear(128, 256),
                                    # nn.ReLU(),
                                    # nn.Linear(256, 128),
                                    # nn.ReLU(),
                                    nn.Linear(128, 6),
                                    nn.Sigmoid())
        

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.lstm(embeddings, (torch.randn(2 * n_layers, len(X_batch), hidden_dim, device = "cuda"), torch.randn(2 * n_layers, len(X_batch), hidden_dim, device = "cuda")))
        return self.linear(output[:,-1])


In [10]:
model = RNNClassifier()

NameError: name 'vocab' is not defined

In [18]:
torch.randn(n_layers, 10, hidden_dim).size()

torch.Size([2, 10, 128])

In [8]:
model = torch.load("lstm_model.pt")
vocab = torch.load("vocab.pt")

In [17]:
valid_loader = torch.utils.data.DataLoader(val_ds, batch_size = batch_size, shuffle=False)

In [17]:
from ignite.metrics import Precision, Accuracy

In [18]:
precision = Precision(is_multilabel=True, device = "cuda")
acc = Accuracy(is_multilabel=True, device = "cuda")

In [20]:
precision.reset()
acc.reset()
for data, label in valid_loader:
    data = data.to("cuda")
    label = label.to("cuda")
    with torch.no_grad():
        pred = model(data)
    pred = torch.where(pred<0.5, 0, 1)
    precision.update((pred, label))
    acc.update((pred, label))
print("Precision: ", precision.compute())
print("Accuracy: ", acc.compute())

Precision:  tensor([0.8839, 0.6273, 0.8830, 0.0000, 0.8016, 0.4364], device='cuda:0',
       dtype=torch.float64)
Accuracy:  0.9509306260575296


In [5]:
max_words = 2000

In [9]:
text_data = "you are a motherfucker"

In [10]:
text_data = vocab(tokenizer(text_data))
text_data = text_data +([0]* (max_words-len(text_data))) if len(text_data)<max_words else text_data[:max_words]
text_data = torch.tensor(text_data, dtype = torch.int32)
text_data = torch.unsqueeze(text_data, 0)
text_data = text_data.to("cuda")

In [38]:
text_data.size()

torch.Size([1, 2000])

In [15]:
pred = torch.where(model(text_data)<0.5, False, True).cpu().numpy()

In [16]:
report = '''Toxic: {}
            Severe Toxic: {}
            Obscene: {}	
            Threat: {}	
            Insult: {}	
            Identity Hate: {}'''.format(pred[0], pred[1], pred[2], pred[3], pred[4], pred[5])

array([[ True, False,  True, False,  True, False]])